# test

In [6]:
import keras
try:
    model
except NameError:
    model = keras.models.load_model('autoencoder.h5')

Using TensorFlow backend.


In [7]:
model.get_layer('binarizer')

In [8]:
encoder = keras.models.Model(inputs=model.input,
                             outputs=model.get_layer('binarizer').output)

In [9]:
encoder.output.shape

TensorShape([Dimension(None), Dimension(1), Dimension(256)])

In [10]:
import os
spec_dir = 'data/spec'
spectrograms = [os.path.join(spec_dir, f) for f in os.listdir(spec_dir)]

In [11]:
import numpy as np
import random
from sklearn.preprocessing import maxabs_scale
def load(f):
    X = np.load(f).items()[0][-1]
    return maxabs_scale(X).transpose()

In [12]:
encoder.predict(load(random.choice(spectrograms)).reshape((1, 512, 184)))

array([[[ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,
          1.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,
          0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,
          1.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,
          1.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
          1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,
          0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,
          0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,
          0.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,
          1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  0.,
          0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
          1.,  1.,  0.,  0.,  1.,  1.,

In [21]:
import itertools

Z = np.array([2**i for i in range(64)], dtype=np.uint64).transpose()

def encode_batch(encoder, files, batch_size):
    # reading all the data at once will likely raise an Exeption.
    # iterate through the files and yield small batches of results
    def gen():
        X = np.zeros((batch_size, 512, 184))
        for i, f in enumerate(files):
            i = i % batch_size
            X[i,:] = load(f)
            if i == batch_size - 1:
                p = encoder.predict(X).astype(np.uint64)
                # have to keep track of two keys since numpy doesn't support 64+ bit integers
                code1 = p[:,:,:64] @ Z
                code2 = p[:,:,64:128] @ Z
                code3 = p[:,:,128:192] @ Z
                code4 = p[:,:,192:256] @ Z
                yield np.array((code1, code2, code3, code4))
                X = np.zeros((batch_size, 512, 184))
    return np.concatenate(list(gen()), axis=1)

In [22]:
len(spectrograms)

208952

In [25]:
%%time
codes = encode_batch(encoder, spectrograms, 500)

predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting

In [2]:
import json
import numpy as np
codes = np.array(json.load(open('codes.json')))

In [3]:
codes = codes[:,:,0]  # why?

IndexError: too many indices for array

In [4]:
codes.shape

(4, 208500)

In [13]:
import pandas as pd
df = pd.DataFrame(codes.transpose(), columns=['key1', 'key2', 'key3', 'key4'])
df['raw_path'] = spectrograms[:208500]

In [14]:
df[['artist', 'album', 'track', 'seconds']] = df.raw_path.str.split('-', 3, expand=True)

In [15]:
df.head()

,key1,key2,key3,key4,raw_path,artist,album,track,seconds
0,2.017826e+18,1.395570e+19,4.700302e+18,1.144438e+19,data/spec/Deep Purple-The Very Best of Deep Pu...,data/spec/Deep Purple,The Very Best of Deep Purple Rhino,Burn.mp3,81.npz
1,1.326642e+19,1.721681e+19,5.026499e+18,1.613927e+19,data/spec/Incubus-If Not Now When-If Not Now W...,data/spec/Incubus,If Not Now When,If Not Now When.mp3,134.npz
2,1.347465e+19,2.457494e+18,3.653526e+18,1.218251e+19,data/spec/Pearl Jam-No Code-Around The Bend.mp...,data/spec/Pearl Jam,No Code,Around The Bend.mp3,203.npz
3,6.893029e+18,1.062149e+19,5.115639e+18,5.540336e+18,data/spec/The Bravery-The Bravery-Aoc.mp3 - 38...,data/spec/The Bravery,The Bravery,Aoc.mp3,38.npz
4,1.281688e+19,5.201920e+18,3.451620e+18,3.202314e+18,data/spec/Creedence Clearwater Revival-Cosmos ...,data/spec/Creedence Clearwater Revival,Cosmos Factory 2008 40th Anniversary Edition,Lookin Out My Back Door.mp3,41.npz


In [16]:
df.drop('raw_path', axis=1, inplace=True)

In [17]:
df['artist'] = df.artist.str.split('/', expand=True)[2]

In [18]:
df['seconds'] = df.seconds.str.split('.', expand=True)[0]

In [19]:
df.head()

,key1,key2,key3,key4,artist,album,track,seconds
0,2.017826e+18,1.395570e+19,4.700302e+18,1.144438e+19,Deep Purple,The Very Best of Deep Purple Rhino,Burn.mp3,81
1,1.326642e+19,1.721681e+19,5.026499e+18,1.613927e+19,Incubus,If Not Now When,If Not Now When.mp3,134
2,1.347465e+19,2.457494e+18,3.653526e+18,1.218251e+19,Pearl Jam,No Code,Around The Bend.mp3,203
3,6.893029e+18,1.062149e+19,5.115639e+18,5.540336e+18,The Bravery,The Bravery,Aoc.mp3,38
4,1.281688e+19,5.201920e+18,3.451620e+18,3.202314e+18,Creedence Clearwater Revival,Cosmos Factory 2008 40th Anniversary Edition,Lookin Out My Back Door.mp3,41


In [20]:
df.to_csv('codes.csv')

In [21]:
# see,
# https://en.wikipedia.org/wiki/Hamming_distance#Algorithm_example
def hamming(n1, n2):
    # this number is made of each bit in either n1 or n2
    # but not both
    v = n1 ^ n2
    d = 0
    while v != 0:
        # subtracting 1 clears the least bit, a, in v and sets all bits
        # before a which are cleared by the logical &
        # 2^n = sum(2^m for 0 <= m <= n-1)
        d += 1
        v &= v - 1
    return d

In [43]:
from functools import partial
def query(key1, key2, key3, key4, df):
    columns = df.columns
    df_copy = df.copy()
    p1 = partial(hamming, key1)
    p2 = partial(hamming, key2)
    p3 = partial(hamming, key3)
    p4 = partial(hamming, key4)
    df_copy['diffs'] = df.key1.apply(p1) \
                       + df.key2.apply(p2) \
                       + df.key3.apply(p3) \
                       + df.key4.apply(p4)
    return df_copy[df_copy.diffs <= 20].sort_values('diffs')

In [47]:
from IPython.display import display
key1, key2, key3, key4, *rest = next(df.sample().itertuples(index=False))
print(rest)
res = query(key1, key2, key3, key4, df)
with pd.option_context('display.max_colwidth', -1):
    display(res)

['The Kinks', 'The BBC Sessions Disc II', 'Track 8.mp3 ', ' 125']


,key1,key2,key3,key4,artist,album,track,seconds,diffs
125559,16118590657180207104,7147697225320455168,1693152819086377472,3095127896552884736,The Kinks,The BBC Sessions Disc II,Track 8.mp3,125,0


# Try cosine similarity

In [446]:
def encode_batch(encoder, files, batch_size):
    # reading all the data at once will likely raise an Exeption.
    # iterate through the files and yield small batches of results
    def gen():
        X = np.zeros((batch_size, 512, 184))
        for i, f in enumerate(files):
            i = i % batch_size
            X[i,:] = load(f)
            if i == batch_size - 1:
                print('predicting')
                yield encoder.predict(X).reshape((batch_size, 128))
                X = np.zeros((batch_size, 512, 184))
    return np.concatenate(list(gen()), axis=0)

In [470]:
codes = encode_batch(encoder, spectrograms, 500)

predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting
predicting

In [471]:
codes.shape

(208500, 128)

In [477]:
np.save('raw_codes.npy', codes)

In [472]:
from sklearn.metrics.pairwise import cosine_similarity

In [473]:
sims = cosine_similarity(codes)
sims

MemoryError: 

In [ ]:
i = random.randint(0, len(spectrograms)-1)
i

In [ ]:
spectrograms[i]

In [ ]:
np.argsort(sims[i])[:5]

In [ ]:
[spectrograms[j] for j in np.argsort(sims[i])[:5]]

In [ ]:
i = random.randint(0, len(spectrograms)-1)
print(spectrograms[i])
[spectrograms[j] for j in np.argsort(sims[i])[:5]]

In [ ]:
i = random.randint(0, len(spectrograms)-1)
print(spectrograms[i])
[spectrograms[j] for j in np.argsort(sims[i])[:5]]

In [ ]:
i = random.randint(0, len(spectrograms)-1)
print(spectrograms[i])
[spectrograms[j] for j in np.argsort(sims[i])[:5]]

In [ ]:
i = random.randint(0, len(spectrograms)-1)
print(spectrograms[i])
[spectrograms[j] for j in np.argsort(sims[i])[:5]]